In [1]:
!pip install "numpy<1.24" --user

import numpy as np

In [2]:
import pandas as pd

c:\Users\Pierre\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
import matplotlib.pyplot as plt
import geopandas as gpd

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 40)

In [4]:
# Collecte des données

data = pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
zones = gpd.read_file('aires.geojson')

In [5]:
possibles = []

for i in ["Crit'Air 2", "Crit'Air 3", "Crit'Air 4", "Crit'Air 5"] :
    possibles.append(("Diesel", i))
for i in ["Crit'Air 1", "Crit'Air 2", "Crit'Air 3"] :
    possibles.append(("Essence", i))
    
possibles.append(("Hybride rechargeable", "Crit'Air 1"))
possibles.append(('Electrique et hydrogène', "Crit'Air E"))

In [59]:
# Parc historique

# On se focalise sur les véhicules particuliers et les camionnettes
df = data.loc[(data["CATEGORIE_VEHICULE"] == "Véhicule particulier") |
                (data["CATEGORIE_VEHICULE"] == "Dérivé VP") |
                (data["CATEGORIE_VEHICULE"] == "Camionnette")]

def age_num(age) :
    res = []
    for i in range (len(age)) :
        res.append(int(str(age[i])[0:2]))
    return res

# On créé une colonne avec l'âge du véhicule sous forme numérique
df.insert(loc = 1, column = 'AGE_NUM', value = age_num(np.array(df['AGE'], dtype = str)))

# On peut se séparer de certaines colonnes
df = df.drop(['REGION_CODE', 'REGION_LIBELLE', 'CLASSE_VEHICULE', 'STATUT_UTILISATEUR', 'AGE'], axis = 1)

# On effectue la somme pour les différentes régions et statuts d'utilisateurs
df = df.groupby(['CATEGORIE_VEHICULE','CARBURANT','CRITAIR', 'AGE_NUM'], as_index = False).sum()
df = df.sort_values(by = ['CATEGORIE_VEHICULE','CARBURANT','CRITAIR', 'AGE_NUM'])

df.head(5)

,CATEGORIE_VEHICULE,CARBURANT,CRITAIR,AGE_NUM,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
0,Camionnette,Diesel,Crit'Air 1,0,0,0,0,0,0,0,0,0,0,1,2,1
1,Camionnette,Diesel,Crit'Air 1,1,4,0,0,0,0,0,0,0,0,0,0,0
2,Camionnette,Diesel,Crit'Air 1,2,0,2,0,0,0,0,0,0,0,0,0,0
3,Camionnette,Diesel,Crit'Air 1,3,0,0,2,0,0,0,0,0,0,0,0,0
4,Camionnette,Diesel,Crit'Air 2,0,12225,133674,240464,252118,257630,265445,291117,318619,334373,354280,298563,317131


In [60]:
# Il faut rajouter les lignes pour les âges de véhicule qui n'existent pas encore

for p in possibles :
    for v in ["Véhicule particulier", "Camionnette", "Dérivé VP"] :
        ages = pd.unique(df.loc[(df['CATEGORIE_VEHICULE'] == v) &
                                (df['CARBURANT'] == p[0]) &
                                (df['CRITAIR'] == p[1])]
                            ['AGE_NUM']
                        )
        for j in range (0, 26) :
            if j not in ages :
                res = [v, p[0], p[1], j]
                res.extend(np.zeros(12, dtype = int))
                res = pd.DataFrame([res], columns = df.columns)
                df = pd.concat([df, res], ignore_index = True)

df = df.sort_values(by = ['CATEGORIE_VEHICULE','CARBURANT','CRITAIR', 'AGE_NUM'])
df = df.reset_index()
df = df.drop(['index'], axis = 1)

In [61]:
# Fonction de pannes

# On suppose qu'on a f(cat_vhl, car, critair, age_num) qui donne proportion de pannes moyennes
def f(cat_vhl, carb, crit, age) :
    return 1/2

def panne_inter(row) :
    return (np.random.binomial(row[df.columns.to_list()[-1]], f(row['CATEGORIE_VEHICULE'], row['CARBURANT'],
                                    row['CRITAIR'], row['AGE_NUM'])))

def panne(df : pd.DataFrame) :
    df['Pannes'] = df.apply(panne_inter, axis = 1)
    return df

# Ca marche!


In [62]:
# Nombre de véhicules neufs à rajouter, à utiliser après pannes

def nbr_vhl_neufs_fonc(df : pd.DataFrame, r) :
    res = df[df.columns.to_list()[-1]].sum() + r*df[df.columns.to_list()[-2]].sum()
    return int(res)

In [63]:
# Distribution des véhicules neufs

def g(cat_vhl, carb, crit, age_num, annee) :
    if age_num == 0 :
        return 0.1
    else :
        return 0

def approx_binomial(n, p) :
    return np.random.normal(n*p, np.sqrt(n*p*(1-p)))

def neufs_inter(row, nbr_vhl_neufs, annee) :
    return (approx_binomial(nbr_vhl_neufs, g(row['CATEGORIE_VEHICULE'], row['CARBURANT'],
                                             row['CRITAIR'], row['AGE_NUM'], annee)))

def vhl_neufs(df : pd.DataFrame, nbr_vhl_neufs, annee) :
    df['Neufs'] = df.apply(neufs_inter, nbr_vhl_neufs = nbr_vhl_neufs, annee = annee, axis = 1)
    return df

# g est la fonction qui donne les proportions de tel ou tel type de véhicule dans les véhicules neufs
# Elle renvoie 0 pour les lignes où l'âge du véhicule est supérieur à 0
        

In [64]:
def final(df : pd.DataFrame, annee) :
    # Dans un premier temps, on remet les +25 qui sont pas en pannes
    df['25_plus'] = pd.DataFrame(np.zeros(len(df)))
    idx = df.loc[df['AGE_NUM'] == 25].index
    df['25_plus'][idx] = df[df.columns.to_list()[-3]][idx] - df['Pannes'][idx]

    # Ensuite, on fait shifter toutes les colonnes qui ne sont pas +25, en sommant sur la colonne créée au-dessus
    df[str(annee)] = df['25_plus']
    idx_2 = df.loc[df['AGE_NUM'] != 25].index
    idx_3 = df.loc[df['AGE_NUM'] != 0].index
    df[str(annee)][idx_3] += (df[df.columns.to_list()[-5]][idx_2] - df['Pannes'][idx_2]).shift(periods = 1, axis = 0)
    
    # Ne pas oublier de rajouter les neufs
    df[str(annee)] += df['Neufs']

    return df

df2 = panne(df)
df2 = vhl_neufs(df2, nbr_vhl_neufs_fonc(df2, 0.005), 2023)
df2 = final(df2, 2023)
df2.head(5)


<ipython-input-64-3d6566e8c114>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['25_plus'][idx] = df[df.columns.to_list()[-3]][idx] - df['Pannes'][idx]
<ipython-input-64-3d6566e8c114>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(annee)][idx_3] += (df[df.columns.to_list()[-5]][idx_2] - df['Pannes'][idx_2]).shift(periods = 1, axis = 0)


,CATEGORIE_VEHICULE,CARBURANT,CRITAIR,AGE_NUM,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022,Pannes,Neufs,25_plus,2023
0,Camionnette,Diesel,Crit'Air 1,0,0,0,0,0,0,0,0,0,0,1,2,1,0,2.243110e+06,0.0,2.243110e+06
1,Camionnette,Diesel,Crit'Air 1,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0.000000e+00,0.0,1.000000e+00
2,Camionnette,Diesel,Crit'Air 1,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0.000000e+00,0.0,0.000000e+00
3,Camionnette,Diesel,Crit'Air 1,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0.000000e+00,0.0,0.000000e+00
4,Camionnette,Diesel,Crit'Air 2,0,12225,133674,240464,252118,257630,265445,291117,318619,334373,354280,298563,317131,158419,2.242742e+06,0.0,2.242742e+06


In [65]:
# Ne pas oublier de drop les colonnes 'Pannes', 'Neufs' et '25_plus'